<a href="https://colab.research.google.com/github/HodaMemar/A3/blob/main/MedSpcy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/mydrive/')
!ls

Mounted at /content/mydrive/
mydrive  sample_data


In [ ]:
!pip install medspacy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 111 kB 7.3 MB/s 
     |████████████████████████████████| 5.9 MB 49.1 MB/s 
     |████████████████████████████████| 69 kB 6.8 MB/s 
     |████████████████████████████████| 71 kB 7.7 MB/s 
     |████████████████████████████████| 558 kB 50.8 MB/s 
     |████████████████████████████████| 236 kB 54.3 MB/s 
     |████████████████████████████████| 235 kB 50.6 MB/s 
     |████████████████████████████████| 1.5 MB 35.5 MB/s 
     |████████████████████████████████| 68 kB 4.8 MB/s 
     |████████████████████████████████| 575 kB 47.9 MB/s 
     |████████████████████████████████| 749 kB 57.6 MB/s 
     |████████████████████████████████| 231 kB 58.1 MB/s 
     |████████████████████████████████| 457 kB 48.8 MB/s 
     |████████████████████████████████| 660 kB 58.2 MB/s 
     |████████████████████████████████| 42 kB 1.3 MB/s 
     |████████████████████████████████| 

In [ ]:
import medspacy
import pandas as pd
nlp = medspacy.load()
sectionizer = nlp.add_pipe("medspacy_sectionizer")

In [ ]:
def nlp_sections(x):
      doc=nlp(x)
      sections = doc._.sections
      return sections

In [ ]:
import re
def preprocess(x):
    y=re.sub('\\[(.*?)\\]','',x) #remove de-identified brackets
    y=re.sub('[0-9]+\.','',y) #remove 1.2. since the segmenter segments based on this
    y=re.sub('dr\.','doctor',y)
    y=re.sub('m\.d\.','md',y)
    y=re.sub('admission date:','',y)
    y=re.sub('Admission Date:','',y)
    y=re.sub('discharge date:','',y)
    y=re.sub('Discharge Date:','',y)
    y=re.sub('Date of Birth:','',y)
    y=re.sub('date of birth:','',y)
    y=re.sub('--|__|==','',y)
    y = y.strip()
    return y

<input>:4: DeprecationWarning: invalid escape sequence \.
<input>:4: DeprecationWarning: invalid escape sequence \.
<input>:5: DeprecationWarning: invalid escape sequence \.
<input>:4: DeprecationWarning: invalid escape sequence \.
<input>:5: DeprecationWarning: invalid escape sequence \.
<input>:6: DeprecationWarning: invalid escape sequence \.
<input>:4: DeprecationWarning: invalid escape sequence \.
<input>:5: DeprecationWarning: invalid escape sequence \.
<input>:6: DeprecationWarning: invalid escape sequence \.
<input>:4: DeprecationWarning: invalid escape sequence \.
<input>:5: DeprecationWarning: invalid escape sequence \.
<input>:6: DeprecationWarning: invalid escape sequence \.
<input>:4: DeprecationWarning: invalid escape sequence \.
<input>:5: DeprecationWarning: invalid escape sequence \.
<input>:6: DeprecationWarning: invalid escape sequence \.
<input>:4: DeprecationWarning: invalid escape sequence \.
<input>:5: DeprecationWarning: invalid escape sequence \.
<input>:6: Dep

In [ ]:

%%time


for chunk_index in (9,13,14) :
    filename="/content/mydrive/MyDrive/PaperA3/Notes/Notes_"+str(chunk_index)+".csv"
    
    df=pd.read_csv(filename)
    print(filename)
    #if (chunk_index>0):
    #  break;
    ls_notes=[]
    for i in range(len(df)):
      print(i)
      print("\n")
      text=df.iloc[i]['TEXT']
      doc=nlp(text)
      sections = doc._.sections
      for j in range(len(sections)):
        mystr=sections[j].body_span.text
        mystr_Wothout_line=preprocess(mystr)
        ls_notes.append([df.iloc[i]['HADM_ID'],df.iloc[i]['DRG_CODE'],
                         df.iloc[i]['GENDER'],df.iloc[i]['AGE'],
                         df.iloc[i]['CHARTDATE'],
                         df.iloc[i]['CATEGORY'],df.iloc[i]['TAA'],
                         sections[j].title_span.text,sections[j].body_span.text,
                         sections[j].category,mystr_Wothout_line])
    df2=pd.DataFrame(ls_notes,columns=["HADM_ID","DRG_CODE",'GENDER','AGE','CHARTDATE','CATEGORY',"TAA",
                                       "title_span","body_span","category_Inner","body_span_Wothout_line"]) 
    filename="section_"+str(chunk_index)+".csv"
    df2.to_csv("/content/mydrive/MyDrive/PaperA3/Section/" + filename)

